# Restaurant Revenue Prediction

## 実際のプログラム

### ライブラリのインポート

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_train = pd.read_csv('../input/restaurant-revenue-prediction/train.csv.zip')
df_test = pd.read_csv('../input/restaurant-revenue-prediction/test.csv.zip')

In [ ]:
print("訓練データの型と欠損値")
print(df_train.info())
print("")
print("テストデータの型と欠損値") 
print(df_test.info())

In [ ]:
# 前処理前
print("説明変数の一覧\n", df_train.columns)    # 説明変数の一覧
print("訓練データの中身\n", df_train)

### データの前処理

In [ ]:
#前処理がしやすい様に、trainとtestを結合
df_whole = pd.concat([df_train, df_test], axis=0)

#Open Dateを年・月・日に分解
df_whole["Open Date"] = pd.to_datetime(df_whole["Open Date"])   # Date型に直す
df_whole["Year"] = df_whole["Open Date"].apply(lambda x:x.year)     # 年
df_whole["Month"] = df_whole["Open Date"].apply(lambda x:x.month)   # 月
df_whole["Day"] = df_whole["Open Date"].apply(lambda x:x.day)       # 日
basedate = df_whole['Open Date'].max()          # データ内の最新の日付を基準にする    
df_whole["Business period"] = (basedate - df_whole['Open Date']).apply(lambda x: x.days)    # 店が開店していた期間


#Cityを数値に変換
le = LabelEncoder()
df_whole["City"] = le.fit_transform(df_whole["City"])

# City Groupを数値に変換 Other -> 0, Big Cities -> 1
df_whole["City Group"] = df_whole["City Group"].map({"Other":0, "Big Cities":1})

#Typeを数値に変換 FC -> 0, IL -> 1, DT -> 2, MB -> 3
df_whole["Type"] = df_whole["Type"].map({"FC":0, "IL":1, "DT":2, "MB":3})

#再びtrainとtestに分割
df_train = df_whole.iloc[:df_train.shape[0]]
df_test = df_whole.iloc[df_train.shape[0]:]

In [ ]:
# 前処理後
print("説明変数の一覧\n", df_train.columns)    # 説明変数の一覧
print("訓練データの中身\n", df_train)

### 相関を見る
- 目的変数の"revenue"との相関を見る
  - 相関が0.2以上あるのは"City","City Group","Bussiness period"
  - P1からP37はあまり相関がないのでそのまま使わなくてもいいかも

In [ ]:
# ヒストグラムの表示
import seaborn as sns

sns.distplot(df_train.revenue)

In [ ]:
# 各項目と'revenue'の相関
import matplotlib.pyplot as plt

plt.figure(figsize=(45,25))
sns.heatmap(df_train.corr(), annot = True, cmap='Blues')

### P1からP37がrevenueとあまり相関がないので全部使うのは無駄かも
- 主成分分析で次元削減

In [ ]:
# P1からP37がrevenueがあまり相関がないので全部使うのは無駄かも
# 主成分分析で次元削減

df_whole = pd.concat([df_train, df_test], axis=0)   # データの結合

import numpy as np
from sklearn.decomposition import PCA

# データの用意
df_whole.columns
X = df_whole.loc[:, ['P1', 'P2', 'P3', 'P4',
       'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15',
       'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35',
       'P36', 'P37']]

# 標準化
# Xに、Xを標準化したデータを代入
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 主成分分析のインスタンスを生成。
pca = PCA(n_components = 3)

# データから変換モデルを学習し、変換する。
X_pca = pca.fit_transform(X)

df_whole['P_1'] = X_pca[:, 0]
df_whole['P_2'] = X_pca[:, 1]
df_whole['P_3'] = X_pca[:, 2]


#再びtrainとtestに分割
df_train = df_whole.iloc[:df_train.shape[0]]
df_test = df_whole.iloc[df_train.shape[0]:]

print(df_train)

In [ ]:
del_P1_P37_columns = [col for col in df_train.columns if col not in ['Id', 'Open Date',
                                                                    'P1', 'P2', 'P3', 'P4','P5', 'P6', 'P7', 'P8', 'P9', 'P10', 
                                                                    'P11', 'P12', 'P13', 'P14', 'P15','P16', 'P17', 'P18', 'P19', 'P20', 
                                                                    'P21', 'P22', 'P23', 'P24', 'P25','P26', 'P27', 'P28', 'P29', 'P30', 'P31', 
                                                                    'P32', 'P33', 'P34', 'P35','P36', 'P37']]

# 相関を見直す
plt.figure(figsize=(45,25))
sns.heatmap(df_train[del_P1_P37_columns].corr(), annot = True, cmap='Blues')

### 学習のためにデータ整形

In [ ]:
#目的変数を抽出
revenue = df_train["revenue"]
del df_train["revenue"]

#学習に使う特徴量を決定
del_train_columns = [col for col in df_train.columns if col not in ['Id', 'Open Date',
                                                                    'P1', 'P2', 'P3', 'P4','P5', 'P6', 'P7', 'P8', 'P9', 'P10', 
                                                                    'P11', 'P12', 'P13', 'P14', 'P15','P16', 'P17', 'P18', 'P19', 'P20', 
                                                                    'P21', 'P22', 'P23', 'P24', 'P25','P26', 'P27', 'P28', 'P29', 'P30', 'P31', 
                                                                    'P32', 'P33', 'P34', 'P35','P36', 'P37']]    # IdとOpen Date,P1-P37は使わない
del_test_columns = [col for col in df_train.columns if col not in ['Id', 'Open Date',
                                                                    'P1', 'P2', 'P3', 'P4','P5', 'P6', 'P7', 'P8', 'P9', 'P10', 
                                                                    'P11', 'P12', 'P13', 'P14', 'P15','P16', 'P17', 'P18', 'P19', 'P20', 
                                                                    'P21', 'P22', 'P23', 'P24', 'P25','P26', 'P27', 'P28', 'P29', 'P30', 'P31', 
                                                                    'P32', 'P33', 'P34', 'P35','P36', 'P37']]    # テストデータも同じく

In [ ]:
print("df_train\n", df_train[del_train_columns])   # 学習データの説明変数 
print("revenue\n", revenue)     # 学習データの目的変数
print("--------------------------------------------------------------------------------------")
print("df_test\n", df_test[del_test_columns])     # テストデータの説明変数(目的変数はなし)

### scikit-learnの回帰手法を比較
- scikit-learnのアルゴリズム・チートシートにある回帰手法を使用
  - チートシート：https://qiita.com/sugulu_Ogawa_ISID/items/e3fc39f2e552f2355209
  

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

names = ['Lasso', 'Ridge', 'ElasticNet', 'SVR linear', 'SVR rbf', 'BaggingRegressor', 'RandomForestRegressor']
models = [Lasso(), Ridge(), ElasticNet(), SVR(kernel='linear'), SVR(kernel='rbf'), BaggingRegressor(), RandomForestRegressor()]

max_score = 0
best_model = 0

for name, model in zip(names, models):
    model.fit(df_train[del_train_columns], revenue)
    score = model.score(df_train[del_train_columns], revenue)
    print("{}: {}".format(name, score))
    if score > best_model:
        best_model = score
        best_model_name = name
print()
print("ベストモデル: {}".format(best_model_name))
print("ベストスコア: {}".format(best_model))

### ランダムフォレスト
- 上の比較で一番良かった
- 決定木を複数作って、それを合わせることでより良いモデルを構築する
  - 詳しくはこのサイトで　https://qiita.com/Hawaii/items/5831e667723b66b46fba

In [ ]:
#ランダムフォレスト
model = RandomForestRegressor()

model.fit(df_train[del_train_columns], revenue)
score = model.score(df_train[del_train_columns], revenue)
print("スコア:", score)

### 予測結果を可視化
- 訓練データで学習した結果を見る
  - テストデータには正解(目的変数)がないため
  - 精度評価の指標はRMSE(二乗平均平方根誤差)  

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
 
#予測値と正解値を描写する関数
def True_Pred_map(pred_df):
    RMSE = np.sqrt(mean_squared_error(pred_df['true'], pred_df['pred']))
    R2 = r2_score(pred_df['true'], pred_df['pred']) 
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.scatter('true', 'pred', data=pred_df)
    ax.set_xlabel('True Value', fontsize=15)
    ax.set_ylabel('Pred Value', fontsize=15)
    ax.set_xlim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    ax.set_ylim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    x = np.linspace(pred_df.min().min()-0.1, pred_df.max().max()+0.1, 2)
    y = x
    ax.plot(x,y,'r-')
    plt.text(0.1, 0.9, 'RMSE = {}'.format(str(round(RMSE, 5))), transform=ax.transAxes, fontsize=15)
    plt.text(0.1, 0.8, 'R^2 = {}'.format(str(round(R2, 5))), transform=ax.transAxes, fontsize=15)

In [ ]:
prediction = model.predict(df_train[del_train_columns])
pred_df = pd.concat([revenue.reset_index(drop=True), pd.Series(prediction)], axis=1)
pred_df.columns = ['true', 'pred']
print("<予測結果>\n", pred_df.head())
RMSE = np.sqrt(mean_squared_error(pred_df['true'], pred_df['pred']))
print("\nRMSE:", RMSE)

In [ ]:
# グラフで可視化(赤線に近いほど正しく予測できている)
"""
ｘ軸：正解値
ｙ軸：予測値
ラベルは1e7(1000万倍)した値
"""
True_Pred_map(pred_df)

### kaggle提出のためにデータを出力

In [ ]:
prediction = model.predict(df_test[del_test_columns])
submission = pd.DataFrame({'Id':df_test.Id, 'Prediction':prediction})
submission.to_csv('submission.csv', index=False)